In [1]:
# Adding all needed imports

import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from keras import layers
from keras.models import Model
#print(tf.keras.__version__)
#print(tf.__version__)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math as mt
import random as rng

from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
dataTrain = np.load("dataFormatted/finished/stTrain.npy", allow_pickle=True)
targetTrain = np.load("dataFormatted/finished/stTrainTarget.npy", allow_pickle=True)


dataVal = np.load("dataFormatted/finished/stVal.npy", allow_pickle=True)
targetVal = np.load("dataFormatted/finished/stValTarget.npy", allow_pickle=True)


print(len(dataTrain[0]))
print(len(targetTrain[0]))
print(len(dataTrain))
print(len(targetTrain))

151
151
80
80


In [3]:
numRows = 30
step = 15



def finalSplit(data, targets, numRows, step):
    div = int(numRows/2)
    splits = []
    splitsTarget = []
    for each in range(len(data)):
        run = data[each]
        target = targets[each]
        for i in range(div, len(target) - div, step):
            split = []
            RUL = 0
            for j in range(-div,div):
                index = i + j
                element = run[index].copy()
                RUL = target[index]
                split.append(element)
            splits.append(split)
            splitsTarget.append(RUL)
            
    return splits, splitsTarget

In [4]:
dataSplitTrain, targetSplitTrain = finalSplit(dataTrain, targetTrain, numRows, step)
dataSplitVal, targetSplitVal = finalSplit(dataVal, targetVal, numRows, step)


print(len(dataSplitTrain))
print(len(dataSplitVal))

711
180


In [5]:
dataSplitTrain = np.array(dataSplitTrain)
targetSplitTrain = np.array(targetSplitTrain)
dataSplitVal = np.array(dataSplitVal)
targetSplitVal = np.array(targetSplitVal)



width = len(dataSplitTrain[0][0])
print(width)

10


In [6]:
visible = layers.Input(shape=(numRows, width,))

cnn = layers.Conv1D(width, 3, activation='relu')(visible)
rnn1 = layers.GRU(150, dropout=0.2)(cnn)
#rnn2 = layers.GRU(150, dropout=0.2)(rnn1)


#rnnX = layers.LSTM(300, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(visible)
#rnnY = layers.LSTM(300, dropout=0.2, recurrent_dropout=0.2)(rnnX)

#combo = layers.concatenate([rnn2,rnnY])
dense = layers.Dense(150, activation='relu')(rnn1)
drop = layers.Dropout(0.2)(dense)
out = layers.Dense(1, activation='linear')(drop)

model = Model(inputs=visible, outputs=out)


print(model.summary())


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30, 10)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 28, 10)            310       
_________________________________________________________________
gru_1 (GRU)                  (None, 150)               72450     
_________________________________________________________________
dense_1 (Dense)              (None, 150)               22650     
_________________________________________________________________
dropout_1 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 151       
Total params: 95,561
Trainable params: 95,561
Non-trainable params: 0
_______________________________________________________

In [7]:
model.compile(optimizer='adam',
              loss='mse', 
              metrics=['mae'])


In [8]:
model.fit(dataSplitTrain, targetSplitTrain, epochs=50, verbose=1, validation_data=(dataSplitVal,targetSplitVal))

Train on 711 samples, validate on 180 samples
Epoch 1/50
711/711 [==============================] - 2s 2ms/step - loss: 4317.0364 - mae: 54.4720 - val_loss: 2352.7849 - val_mae: 39.7267
Epoch 2/50
711/711 [==============================] - 1s 1ms/step - loss: 1715.6021 - mae: 34.9186 - val_loss: 1520.9804 - val_mae: 33.8449
Epoch 3/50
711/711 [==============================] - 1s 1ms/step - loss: 1456.0091 - mae: 32.9710 - val_loss: 1477.6971 - val_mae: 33.1152
Epoch 4/50
711/711 [==============================] - 1s 1ms/step - loss: 1450.8732 - mae: 32.9265 - val_loss: 1419.2475 - val_mae: 32.5236
Epoch 5/50
711/711 [==============================] - 1s 1ms/step - loss: 1330.1152 - mae: 31.5563 - val_loss: 1209.5934 - val_mae: 30.3483
Epoch 6/50
711/711 [==============================] - 1s 1ms/step - loss: 956.2823 - mae: 26.4377 - val_loss: 739.3863 - val_mae: 22.0578
Epoch 7/50
711/711 [==============================] - 1s 1ms/step - loss: 542.4508 - mae: 19.1961 - val_loss: 460.84

In [ ]:
model_json = model.to_json()
with open("./model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


In [13]:
np.save("./valData.npy", simsTest)
np.save("./valTarget.npy", targetTest)